# Subsequent Memory Effect

### Imports

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


### Settings

In [2]:
# Paths
PROJECT_PATH = 'C:/Users/micha/projects/oscillation_vs_exponent/'


### Load data

In [3]:
# load spectral parameterization results
df = pd.read_csv(f"{PROJECT_PATH}/data/results/spectral_parameters.csv", index_col=0)

# log transform power values
for var in ['alpha_bp','alpha_adj']:
    df[var] = df[var].apply(np.log10)
df['alpha_adj'].loc[np.isnan(df['alpha_adj'])] = 0 # set NaN to zero (negative power values due to noise/error in model fit)

# set Nan values to zero (no detectable alpha peak)
df['alpha_pw'].loc[np.isnan(df['alpha_pw'])] = 0 # set NaN to zero (negative power values due to noise/error in model fit)

# show
print(df.shape)
print(len(df)/8)
df.head()

(5560, 19)
695.0


C:\Users\micha\AppData\Local\Temp\ipykernel_42924\2329723685.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['alpha_adj'].loc[np.isnan(df['alpha_adj'])] = 0 # set NaN to zero (negative power values due to noise/error in model fit)
C:\Users\micha\AppData\Local\Temp\ipykernel_42924\2329723685.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['alpha_pw'].loc[np.isnan(df['alpha_pw'])] = 0 # set NaN to zero (negative power values due to noise/error in model fit)


,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,material,memory,epoch,offset,knee,exponent,f_rotation,alpha_cf,alpha_pw,alpha_bw,alpha_bp,alpha_adj
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,words,hit,pre,8.563988,12.837555,3.622099,NaN,NaN,0.000000,NaN,4.415105,2.448049
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,words,hit,pre,7.536717,10.589396,3.001966,14.801580,11.496755,0.137561,2.496898,4.207266,0.000000
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,words,hit,pre,7.361091,9.891271,2.916053,19.981929,NaN,0.000000,NaN,4.175968,0.000000
3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,words,hit,pre,7.631240,10.842178,2.951918,38.719363,NaN,0.000000,NaN,4.364474,3.021621
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,words,hit,pre,7.359698,9.714143,2.832629,42.246409,11.739647,0.189450,3.076109,4.324871,3.386532


In [4]:
# compute stimulus-evoked change in each parameter
df_pre = df.loc[df['epoch']=='pre']
df_post = df.loc[df['epoch']=='post']

# compute change
columns = df_pre.columns.tolist()
df_diff = df_pre.loc[:, columns[:10]]
df_diff.loc[:, columns[10:]] = df_post.loc[:, columns[10:]].values - df_pre.loc[:, columns[10:]].values
df_diff.loc[:,'f_rotation'] = df_pre.loc[:,'f_rotation']
df_diff.loc[:,'alpha_bp'] = df_post.loc[:,'alpha_bp'].values - df_pre.loc[:,'alpha_bp'].values
df_diff.loc[:,'alpha_adj'] = df_post.loc[:,'alpha_adj'].values - df_pre.loc[:,'alpha_adj'].values
df_diff

,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,material,memory,epoch,offset,knee,exponent,f_rotation,alpha_cf,alpha_pw,alpha_bw,alpha_bp,alpha_adj
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,words,hit,pre,-0.071111,-1.195204,-0.057229,NaN,NaN,0.000000,NaN,0.079520,-2.448049
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,words,hit,pre,-0.136333,-2.320040,-0.062502,14.801580,-3.198155,0.022115,-0.164309,0.182334,3.621966
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,words,hit,pre,-0.099353,0.111838,-0.079826,19.981929,NaN,0.000000,NaN,-0.002648,2.885900
3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,words,hit,pre,-0.486147,-2.168077,-0.304933,38.719363,NaN,0.000000,NaN,-0.071258,-3.021621
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,words,hit,pre,-0.352887,-2.043725,-0.215864,42.246409,NaN,-0.189450,NaN,-0.117964,-3.386532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4860,pat22,48,MOF_03-MOF_04,36.00000,-14.00000,3.00000,pat22_48,faces,miss,pre,0.054411,-2.115757,0.013564,NaN,NaN,0.000000,NaN,0.108833,1.178184
4861,pat22,49,MOF_04-MOF_07,40.50000,-14.00000,9.50000,pat22_49,faces,miss,pre,0.188201,-0.545645,0.090283,NaN,NaN,0.000000,NaN,0.053286,-1.580651
4862,pat22,50,MOF_07-MOF_08,44.50000,-14.00000,16.50000,pat22_50,faces,miss,pre,-0.290452,-4.424308,-0.156698,69.801459,NaN,0.000000,NaN,0.128077,0.000000
4863,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000,pat22_51,faces,miss,pre,-0.103320,-4.084912,-0.104189,NaN,NaN,0.000000,NaN,0.178777,-1.543987


In [5]:
# marge with task-modulation results

# load task-modulation results
df_tm = pd.read_csv(r"C:\Users\micha\projects\oscillation_vs_exponent\data\results\ieeg_modulated_channels.csv", index_col=0)

# join
df_merge = pd.merge(df_diff, df_tm, on=['patient','chan_idx','material','memory'])
df_merge

df_merge

,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,material,memory,epoch,...,f_rotation,alpha_cf,alpha_pw,alpha_bw,alpha_bp,alpha_adj,p_val,sign,sig_tm,sig
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,words,hit,pre,...,NaN,NaN,0.000000,NaN,0.079520,-2.448049,0.4358,-1,False,False
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,words,hit,pre,...,14.801580,-3.198155,0.022115,-0.164309,0.182334,3.621966,0.0430,-1,True,True
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,words,hit,pre,...,19.981929,NaN,0.000000,NaN,-0.002648,2.885900,0.8368,1,False,False
3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,words,hit,pre,...,38.719363,NaN,0.000000,NaN,-0.071258,-3.021621,0.0032,1,True,False
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,words,hit,pre,...,42.246409,NaN,-0.189450,NaN,-0.117964,-3.386532,0.0004,1,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2775,pat22,48,MOF_03-MOF_04,36.00000,-14.00000,3.00000,pat22_48,faces,miss,pre,...,NaN,NaN,0.000000,NaN,0.108833,1.178184,0.0654,-1,False,False
2776,pat22,49,MOF_04-MOF_07,40.50000,-14.00000,9.50000,pat22_49,faces,miss,pre,...,NaN,NaN,0.000000,NaN,0.053286,-1.580651,0.2374,-1,False,False
2777,pat22,50,MOF_07-MOF_08,44.50000,-14.00000,16.50000,pat22_50,faces,miss,pre,...,69.801459,NaN,0.000000,NaN,0.128077,0.000000,0.0038,-1,True,False
2778,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000,pat22_51,faces,miss,pre,...,NaN,NaN,0.000000,NaN,0.178777,-1.543987,0.0004,-1,True,True


### Run 2x2 Anova

In [6]:
# drop missing data channels
df_merge['drop'] = False

for i_row, row in df_merge.iterrows():
    if np.isnan(row['exponent']):
        df_merge.loc[((df_merge['patient']==row['patient']) & (df_merge['chan_idx']==row['chan_idx'])), 'drop'] = True

df_clean = df_merge.loc[~df_merge['drop']]
print(f"NaN channels to drop: {int(np.sum(df_merge['drop'])/4)}")

NaN channels to drop: 35


In [7]:
# get tak-modulated channels
df_tm = df_clean.loc[df_clean['sig']]
print(f"N task-modulated channels: \t{int(len(df_tm)/4)}")


N task-modulated channels: 	112


In [8]:
# run 2-way anova (memory x material)

# loop over variables of interest
results = {}
for var in ['exponent', 'alpha_pw', 'alpha_bp', 'alpha_adj']:
    # repeated measures anova
    results_i = sm.stats.AnovaRM(df_tm, f"{var}", 'unique_id', within=['memory', 'material']).fit()

    # aggregate
    results[var] = results_i

    # show results
    print(f"\n\n===================  {var}  ===================\n")
    print(results_i)



===================  exponent  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.0190 1.0000 111.0000 0.8907
material         6.2508 1.0000 111.0000 0.0139
memory:material  0.5927 1.0000 111.0000 0.4430



===================  alpha_pw  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.0006 1.0000 111.0000 0.9805
material         2.5320 1.0000 111.0000 0.1144
memory:material  4.4596 1.0000 111.0000 0.0369



===================  alpha_bp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           4.3642 1.0000 111.0000 0.0390
material         0.3586 1.0000 111.0000 0.5505
memory:material  0.2027 1.0000 111.0000 0.6535



===================  alpha_adj  ==================

In [9]:
df_mean = df.groupby(by=['material','memory']).mean()
df_mean.loc[:, ['exponent', 'alpha_pw', 'alpha_bp', 'alpha_adj']]

exponent  alpha_pw  alpha_bp  alpha_adj
material memory                                         
faces    hit     2.997758  0.094416  3.844581   1.689317
         miss    3.007861  0.092862  3.846836   1.740442
words    hit     3.025198  0.092895  3.854637   1.710122
         miss    3.036344  0.097234  3.877138   1.897580